# Chat with Sponge Bob Square Pants
GUI with Gradio library and system prompt set to mimic Sponge Bob Square Pants talking style. The chat opens in browser (localhost), and the model is Ollama local model
![chat.png](chat.png)

In [25]:
import ollama
import gradio as gr
import os

MODEL = "llama3.2:3b"

First let's set theme using CSS

In [26]:
notebook_dir = os.getcwd()  # Gets the notebook's directory
background_path = os.path.join(notebook_dir, "background.png")

spongebob_theme = gr.themes.Base(
    primary_hue=gr.themes.colors.yellow,
    secondary_hue=gr.themes.colors.blue,
    neutral_hue=gr.themes.colors.green,
    font=["Comic Sans MS", "Arial", "sans-serif"],
    font_mono=["Comic Sans MS", "Arial", "sans-serif"],
    text_size="lg",
    radius_size="lg",  # Rounded corners for bubbly look
)

spongebob_css = f"""
/* Style the chat container */
.gradio-container {{
    background: url('file:///{background_path.replace(os.sep, "/")}') no-repeat center center fixed;
    background-size: cover;
    font-family: 'Comic Sans MS', sans-serif;
}}

/* Style the chatbot message bubbles */
.chatbot .message.user {{
    background-color: #FFFF00; /* SpongeBob yellow */
    color: #000000;
    border-radius: 15px;
    padding: 10px;
    margin: 5px;
}}

.chatbot .message.bot {{
    background-color: #00B7EB; /* Ocean blue */
    color: #FFFFFF;
    border-radius: 15px;
    padding: 10px;
    margin: 5px;
}}

/* Style the input textbox */
input[type="text"] {{
    background-color: #FF69B4; /* Patrick pink */
    color: #FFFFFF;
    border: 2px solid #00FF00; /* Seaweed green border */
    border-radius: 10px;
    padding: 8px;
}}

/* Style buttons */
button {{
    background-color: #00FF00; /* Seaweed green */
    color: #000000;
    border-radius: 20px;
    padding: 10px 20px;
    font-size: 16px;
    font-weight: bold;
    border: 2px solid #FFFF00; /* Yellow border */
}}

/* Style the clear button */
.clear-button {{
    background-color: #FF4500; /* Chum Bucket orange */
    color: #FFFFFF;
}}

/* Style the chat container */
.chatbot {{
    border: 3px solid #00B7EB; /* Ocean blue border */
    border-radius: 20px;
    background-color: rgba(255, 255, 255, 0.2); /* Semi-transparent white */
    backdrop-filter: blur(5px); /* Blurry underwater effect */
}}
"""

Functions to handle the conversation:

In [27]:
SYSTEM_PROMPT = """
You are SpongeBob SquarePants, a cheerful, optimistic sea sponge from Bikini Bottom. Respond to simple or casual questions with enthusiasm, humor, and nautical puns, using phrases like "barnacles!", "tartar sauce!", or "I'm ready!" Incorporate references to Krusty Krab, Patrick, or Squidward. For serious or complex questions, avoid answering directly and instead reply with a funny, evasive phrase like "Let's flip some patties instead!" or "Whoa, that's deeper than the Mariana Trench!" Keep responses short, goofy, and true to SpongeBob's cartoonish voice.

Examples:
- User: What's your favorite food?
  Response: Krabby Patties, of course! *giggles* I flip 'em all day at the Krusty Krab! What's your favorite, buddy?
- User: What's the meaning of life?
  Response: Barnacles, that's a deep one! How 'bout we go jellyfishing with Patrick instead?
"""


def spongebob_chat(message, history):
    messages = [{"role": "system", "content": SYSTEM_PROMPT}] + history + [{"role": "user", "content": message}]
    stream = ollama.chat(
        model=MODEL,
        messages=messages,
        stream=True
    )
    response = ""
    for chunk in stream:
        chunk_content = chunk['message']['content'] or ''
        response += chunk_content.replace("```", "").replace("markdown", "")
        yield response

def add_message(history, message):
    if not history:
        history = []
    # Append user message immediately
    history.append({"role": "user", "content": message})
    yield history, ""  # Show user message and clear textbox

    # Stream assistant response
    response = ""
    for partial_response in spongebob_chat(message, history[:-1]):
        response = partial_response
        yield history + [{"role": "assistant", "content": response}], ""
    history.append({"role": "assistant", "content": response})
    yield history, ""

# Regenerate last response
def regenerate(history):
    if len(history) >= 2 and history[-2]["role"] == "user":
        last_user_msg = history[-2]["content"]
        history = history[:-1]
        response = ""
        for partial_response in spongebob_chat(last_user_msg, history[:-1]):
            response = partial_response
            yield history + [{"role": "assistant", "content": response}], ""
        history.append({"role": "assistant", "content": response})
        yield history, ""
    else:
        yield history, ""

# Clear chat
def clear_chat():
    return [], ""

with gr.Blocks(theme=spongebob_theme, css=spongebob_css) as view:
    gr.Markdown("# SpongeBob's Bikini Bottom Chat! 🧽🌊")

    chatbot = gr.Chatbot(type="messages", height=500)
    msg = gr.Textbox(placeholder="Talk to SpongeBob!", label="Your Message")
    with gr.Row():
        submit_btn = gr.Button("Send to SpongeBob!", variant="primary")
        retry_btn = gr.Button("Retry Last!", variant="secondary")
        clear_btn = gr.Button("Clear Chat!", elem_classes=["clear-button"])

    # Events
    submit_btn.click(add_message, [chatbot, msg], [chatbot, msg])
    retry_btn.click(regenerate, chatbot, [chatbot, msg])
    clear_btn.click(clear_chat, None, [chatbot, msg])
    msg.submit(add_message, [chatbot, msg], [chatbot, msg])

Main call:

In [28]:
view.launch()

* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.
